<a href="https://colab.research.google.com/github/DanielHolzwart/Summarization-with-BART/blob/main/Creating_CV_summaries_with_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


PyMUPDF is used to read in a pdf file and export a text which. BART can then read in tokes from those text files and summarize the content

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 58.0 MB/s eta 0:00:00


In [3]:
import fitz  # PyMuPDF
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap

In [4]:
def extract_text_from_pdf(pdf_path):

    #read in document
    doc = fitz.open(pdf_path)

    #declare empty string
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        #add page to empty string
        text += page.get_text()
    doc.close()
    return text

facebook/large-cnn-bart is one possible choice of transformer models for summarization.

In [5]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


We are looking at a example CV pdf containing
  - CV definition and introduction
  - CV samples of Rachel Green and Juan Garcia



In [38]:
import os
cv_path = os.getcwd() + '/cvsamples.pdf'
cv_text = extract_text_from_pdf(cv_path)

In [62]:
print(cv_text[:1000])

217-333-4610 | GradCareers@illinois.edu | grad.illinois.edu/CareerDevelopment
Curriculum Vitae 
Tips and Samples 
THE BASICS 
The curriculum vitae, also known as a CV or vita, is a comprehensive statement of your 
educational background, teaching, and research experience. It is the standard representation 
of credentials within academia. 

The full CV is only used when applying for academic positions in four-year institutions.

Do not use a CV when applying to community colleges—use a teacher-focused
résumé instead.

Tailor your CV to the specific positions to which you are applying and place more
relevant sections earlier in the document.
–
For a position at a teaching-focused liberal arts college, the CV will strongly
emphasize teaching.
–
For a position at a research-intensive university, the CV will accentuate
research.

Format can vary by field, so also seek disciplinary-specific advice from advisers,
professors, and others within your field.

There are no length restrictions

BART can only process 1024 tokens and a pdf with 8 pages has more than that. Thus, we will split the text into chunks. With max word length of 1024

In [34]:
def split_text_into_chunks(text, max_length=1024):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(current_chunk) + len(word) + 1 > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

Let us check how many chunks our 6 page pdf has.

In [42]:
cv_text_chunks = split_text_into_chunks(cv_text)
print(len(cv_text_chunks))

3


The following function reads in a text, tokenizes it and after feeding it into the BART model exports a sumamry

In [43]:
def generate_summary(text):
    # input text to summarize
    input_text = f"Summarize the following text: {text}"

    # tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # generate the summary
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)

    # decode the output to text
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

Let us check how well that works. As the 3 main topic are 1. CV definition, 2. sample CV Rachel Green & 3. sample CV Juan Garcia. We would expect summarizes similar to these 3 topic after evaluation the 3 chunks. However, the split_text_into_chunks function gives a hard cut and e.g.. chunk 2 will contain information about all three topic. Nevertheless, let us check out what happens

In [44]:
summaries = []
for chunk in cv_text_chunks:
    summaries.append(generate_summary(chunk))

In [52]:
import textwrap
for summary in summaries:
    print('\n'.join(textwrap.wrap(summary, width=80)))
    print('')

The curriculum vitae, also known as a CV or vita, is a comprehensive statement
of your educational background, teaching, and research experience. Tailor your
CV to the specific positions to which you are applying and place more relevant
sections earlier in the document. Use the format appropriate to your particular
academic discipline for a clean look.

Assistant 20xx-20xx Department of English, University of Illinois at Urbana-
Champaign. Research Assistant 20xx to Professor Robert Warren, conducting
primary and secondary source research. Managing Editor, Southern Literary
Journal. Graduate Mentor, Counselor to minority undergraduates, Graduate College
Conference Travel Grant. Graduate College Dissertation Completion Award.

Juan Garcia: Mentored two undergraduate students in data collection and analysis
to visualize the properties of various geotechnical materials. Guided the
students in preparation and presentation of research findings. Graduated with
honors and ranking 4th among en

It looks like the model performed fairly well for the CV introduction and CV sample of Juan Garcia. It looks like text chunk 2 has no reference to Rachel Green. Let us slightly adjust the generate_summary function by introduction a questions to the context and see if BART perfoms better

In [54]:
def generate_summary_with_question(question,text):
    # input text to summarize
    input_text = f"{question} in the following context: {text}"

    # tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # generate the summary
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)

    # decode the output to text
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

In [59]:
question = "Who is Rachel Green?"
summary_rachel= generate_summary_with_question(question, cv_text_chunks[1])
print('\n'.join(textwrap.wrap(summary_rachel, width=80)))

Rachel Green is an assistant professor of English at the University of Illinois
at Urbana-Champaign. She is also the managing editor of the Southern Literary
Journal. Green has been recognized for her work by the U.S. Department of
Education, the American Literature Association, and Butler University.
